# Notebook 1 — Classificação (Modelos Supervisionados)
## Projeto IA — Mundial (World Cups + FIFA Ranking)

**Grupo:** G02  
**Autores:** António Ferreira (nº 9657), Mafalda Barão (nº 20446), Ruben Dias (nº 23033), Gonçalo Gomes (nº 23039), João Morais (nº 23041)  
**Docente:** Rui Fernandes  
**Data:** 2025-12-28  

**Objetivo:** Construir e avaliar modelos de **classificação** com base em dados do Mundial e do ranking FIFA, comparando 2–3 algoritmos e efetuando **otimização de hiperparâmetros**, de forma a obter um modelo com desempenho robusto e interpretável.


### Dados e fontes

Este projeto usa **apenas dados públicos** de futebol:

- **World Cups / Matches** (`WorldCups.csv`, `WorldCupMatches.csv`) — Kaggle: https://www.kaggle.com/datasets/abecklas/fifa-world-cup
- **FIFA World Ranking** (`fifa_ranking-2024-06-20.csv`) — Kaggle: https://www.kaggle.com/datasets/cashncarry/fifaworldranking

> Nota: os ficheiros CSV estão na pasta `/data` do repositório, para execução local sem alteração de caminhos.


## 1) Imports

Importação das bibliotecas necessárias (pandas/numpy e scikit‑learn) e configuração inicial para garantir reprodutibilidade.


In [ ]:
# =========================================================
# Notebook 1 — Classificação + Probabilidade de Campeão
# (Jogos World Cup + FIFA ranking + flag "já foi campeão")
# =========================================================

# --------------------------
# 1) Imports
# --------------------------
import numpy as np
import pandas as pd
import re
from pathlib import Path

from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, log_loss

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import joblib

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 2) Helpers

Funções auxiliares para normalização de nomes de seleções, procura de colunas com nomes diferentes e utilitários de limpeza/preparação.


In [ ]:
# --------------------------
# 2) Helpers
# --------------------------
def norm_team_name(x) -> str:
    if pd.isna(x):
        return x
    return str(x).strip()

def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Normaliza nomes de colunas: remove espaços extra e limpa BOM/variações."""
    df = df.copy()
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    return df

def find_col(df: pd.DataFrame, candidates):
    """Devolve a primeira coluna existente (case-insensitive) dentro de candidates.
    Tolerante a espaços extra e variações simples."""
    def norm(s):
        return re.sub(r"\s+", " ", str(s)).strip().lower()

    lower_map = {norm(c): c for c in df.columns}
    for cand in candidates:
        if cand is None:
            continue
        k = norm(cand)
        if k in lower_map:
            return lower_map[k]
    return None

def safe_to_datetime(s):
    return pd.to_datetime(s, errors="coerce")

def find_data_dir() -> Path:
    """Procura a pasta 'data' a partir do diretório atual (e pais)."""
    p = Path.cwd().resolve()
    for parent in [p, *p.parents]:
        d = parent / "data"
        if d.exists():
            return d
    raise FileNotFoundError("Não encontrei a pasta 'data'. Confirma a estrutura do projeto.")


## 3) Ler datasets

Leitura dos datasets usados no projeto (jogos do Mundial, edições do Mundial e ranking FIFA) a partir da pasta `data/`.


In [ ]:
# --------------------------
# 3) Ler datasets
# --------------------------
DATA_DIR = find_data_dir()

FIFA_PATH = DATA_DIR / "fifa_ranking-2024-06-20.csv"
MATCHES_PATH = DATA_DIR / "WorldCupMatches.csv"
CUPS_PATH = DATA_DIR / "WorldCups.csv"

# encoding='utf-8-sig' ajuda quando o CSV tem BOM no cabeçalho
fifa = pd.read_csv(FIFA_PATH, encoding="utf-8-sig")
matches = pd.read_csv(MATCHES_PATH, encoding="utf-8-sig")
cups = pd.read_csv(CUPS_PATH, encoding="utf-8-sig")

# Normalizar nomes das colunas (evita falhas no find_col por espaços/BOM)
fifa = clean_columns(fifa)
matches = clean_columns(matches)
cups = clean_columns(cups)

print("Diretório de execução:", Path.cwd())
print("DATA_DIR:", DATA_DIR)
print("Ficheiros carregados:", FIFA_PATH.name, MATCHES_PATH.name, CUPS_PATH.name)


Diretório de execução: c:\IA\IA25_P02_G02-main
DATA_DIR: C:\IA\IA25_P02_G02-main\data
Ficheiros carregados: fifa_ranking-2024-06-20.csv WorldCupMatches.csv WorldCups.csv


## 4) Detectar colunas e normalizar

Deteção automática de colunas relevantes e normalização: nomes de seleções, tipos de dados (datas/números) e tratamento de valores em falta.


In [ ]:
# --------------------------
# 4) Detectar colunas e normalizar
# --------------------------
# FIFA
fifa_country_col = find_col(fifa, ["country_full", "country", "team", "team name", "name", "pais"])
fifa_rank_col = find_col(fifa, ["rank", "ranking", "position"])
fifa_points_col = find_col(fifa, ["total_points", "points"])
fifa_date_col = find_col(fifa, ["rank_date", "ranking_date", "date"])  # se existir

if fifa_country_col is None or fifa_rank_col is None:
    raise ValueError("No fifa_ranking.csv não encontrei colunas de país e rank.")

fifa[fifa_country_col] = fifa[fifa_country_col].apply(norm_team_name)
fifa[fifa_rank_col] = pd.to_numeric(fifa[fifa_rank_col], errors="coerce")

if fifa_points_col:
    fifa[fifa_points_col] = pd.to_numeric(fifa[fifa_points_col], errors="coerce")

if fifa_date_col:
    fifa[fifa_date_col] = safe_to_datetime(fifa[fifa_date_col])

# Matches (colunas que disseste que tens)
home_col = find_col(matches, ["Home Team Name"])
away_col = find_col(matches, ["Away Team Name"])
hgoals_col = find_col(matches, ["Home Team Goals"])
agoals_col = find_col(matches, ["Away Team Goals"])
stage_col = find_col(matches, ["Stage"])
year_col = find_col(matches, ["Year", "year", "Edition"])
date_col = find_col(matches, ["Datetime", "Date", "date", "match_date"])

for c in [home_col, away_col, hgoals_col, agoals_col, stage_col]:
    if c is None:
        raise ValueError("No worldcupmatches.csv faltam colunas esperadas (Stage/Home/Away/Goals).")

matches[home_col] = matches[home_col].apply(norm_team_name)
matches[away_col] = matches[away_col].apply(norm_team_name)

if date_col:
    matches[date_col] = safe_to_datetime(matches[date_col])
if year_col:
    matches[year_col] = pd.to_numeric(matches[year_col], errors="coerce").astype("Int64")

# Cups
winner_col = find_col(cups, ["Winner", "winner", "Champion", "champion"])
cup_year_col = find_col(cups, ["Year", "year", "Edition"])

if winner_col is None:
    raise ValueError("No worldcups.csv não encontrei a coluna Winner/Champion. Confirma o nome.")

cups[winner_col] = cups[winner_col].apply(norm_team_name)
if cup_year_col:
    cups[cup_year_col] = pd.to_numeric(cups[cup_year_col], errors="coerce").astype("Int64")

print("Colunas detetadas:")
print("FIFA:", fifa_country_col, fifa_rank_col, fifa_points_col, fifa_date_col)
print("Matches:", home_col, away_col, hgoals_col, agoals_col, stage_col, year_col, date_col)
print("Cups:", winner_col, cup_year_col)


Colunas detetadas:
FIFA: country_full rank total_points rank_date
Matches: Home Team Name Away Team Name Home Team Goals Away Team Goals Stage Year Datetime
Cups: Winner Year


## 5) Criar target y (resultado do jogo) + base features

Criação do alvo `y` (classe do resultado do jogo) e construção das features base a partir dos jogos (ex.: golos, diferenças, contexto do jogo).


In [ ]:
# --------------------------
# 5) Criar target y (resultado do jogo) + base features
# --------------------------
# Limpar linhas sem golos
matches = matches.dropna(subset=[hgoals_col, agoals_col, home_col, away_col, stage_col]).copy()

matches[hgoals_col] = pd.to_numeric(matches[hgoals_col], errors="coerce")
matches[agoals_col] = pd.to_numeric(matches[agoals_col], errors="coerce")
matches = matches.dropna(subset=[hgoals_col, agoals_col]).copy()

# Target: 0=AwayWin, 1=Draw, 2=HomeWin
goal_diff = matches[hgoals_col] - matches[agoals_col]
matches["y"] = np.where(goal_diff > 0, 2, np.where(goal_diff < 0, 0, 1))

matches["stage"] = matches[stage_col].astype(str).str.strip()

# Equipas (para merge e para flags)
matches["home_team"] = matches[home_col]
matches["away_team"] = matches[away_col]

print("Distribuição do target (0=Away,1=Draw,2=Home):")
print(pd.Series(matches["y"]).value_counts(normalize=True).sort_index())


Distribuição do target (0=Away,1=Draw,2=Home):
y
0    0.204225
1    0.223005
2    0.572770
Name: proportion, dtype: float64


## 6) Feature: "já foi campeão?"

Engenharia de features: criação da variável binária **"já foi campeão"** com base no histórico de vencedores do Mundial.


In [ ]:
# --------------------------
# 6) Feature: "já foi campeão?"
# (Se houver ano nos jogos e no cups, usamos "antes do ano do jogo")
# --------------------------
champions_ever = set(cups[winner_col].dropna().unique().tolist())

if cup_year_col is not None:
    champion_year_map = cups[[cup_year_col, winner_col]].dropna().copy()
else:
    champion_year_map = None

def champion_before_year(team: str, year: int) -> int:
    if champion_year_map is None or pd.isna(year):
        return int(team in champions_ever)
    wins = champion_year_map[(champion_year_map[winner_col] == team) & (champion_year_map[cup_year_col] < year)]
    return int(len(wins) > 0)

matches["home_champion_ever"] = matches["home_team"].apply(lambda t: int(t in champions_ever))
matches["away_champion_ever"] = matches["away_team"].apply(lambda t: int(t in champions_ever))

if year_col is not None and champion_year_map is not None:
    matches["home_champion_before"] = matches.apply(
        lambda r: champion_before_year(r["home_team"], int(r[year_col]) if not pd.isna(r[year_col]) else np.nan), axis=1
    )
    matches["away_champion_before"] = matches.apply(
        lambda r: champion_before_year(r["away_team"], int(r[year_col]) if not pd.isna(r[year_col]) else np.nan), axis=1
    )
else:
    matches["home_champion_before"] = matches["home_champion_ever"]
    matches["away_champion_before"] = matches["away_champion_ever"]

display(matches[["home_team","away_team","home_champion_before","away_champion_before","stage","y"]].head(10))


,home_team,away_team,home_champion_before,away_champion_before,stage,y
0,France,Mexico,0,0,Group 1,2
1,USA,Belgium,0,0,Group 4,2
2,Yugoslavia,Brazil,0,0,Group 2,2
3,Romania,Peru,0,0,Group 3,2
4,Argentina,France,0,0,Group 1,2
5,Chile,Mexico,0,0,Group 1,2
6,Yugoslavia,Bolivia,0,0,Group 2,2
7,USA,Paraguay,0,0,Group 4,2
8,Uruguay,Peru,0,0,Group 3,2
9,Chile,France,0,0,Group 1,2


## 7) Preparar ranking FIFA (snapshot ou as-of se tiver data)

Preparação do ranking FIFA (snapshot ou *as-of* por data/ano) e junção do ranking às observações (team, year).


In [ ]:
# --------------------------
# 7) Preparar ranking FIFA (snapshot ou as-of se tiver data)
# --------------------------
import numpy as np
import pandas as pd

# 7.0) Remover colunas duplicadas (antes de renomear)
matches = matches.loc[:, ~matches.columns.duplicated()].copy()

# 7.0.1) Garantir nomes canónicos no matches (home_team, away_team, stage, year)
rename_map = {}
if home_col is not None and home_col != "home_team":
    rename_map[home_col] = "home_team"
if away_col is not None and away_col != "away_team":
    rename_map[away_col] = "away_team"
if stage_col is not None and stage_col != "stage":
    rename_map[stage_col] = "stage"
if year_col is not None and year_col != "year":
    rename_map[year_col] = "year"

if rename_map:
    matches = matches.rename(columns=rename_map)

# 7.0.2) Remover colunas duplicadas (depois de renomear) — ISTO resolve a maioria dos casos
matches = matches.loc[:, ~matches.columns.duplicated()].copy()

# Debug opcional (podes apagar)
dups = matches.columns[matches.columns.duplicated()].tolist()
print("Colunas duplicadas após rename:", dups)

# Validar presença das colunas principais
for c in ["home_team", "away_team"]:
    if c not in matches.columns:
        raise KeyError(f"Falta a coluna '{c}' em matches. Colunas atuais: {list(matches.columns)[:40]} ...")

# 7.1) Tabela FIFA simplificada
cols = [fifa_country_col, fifa_rank_col]
if fifa_points_col:
    cols.append(fifa_points_col)
if fifa_date_col:
    cols.append(fifa_date_col)

fifa_small = fifa[cols].dropna(subset=[fifa_country_col, fifa_rank_col]).copy()
fifa_small = fifa_small.rename(columns={
    fifa_country_col: "team",
    fifa_rank_col: "fifa_rank"
})
if fifa_points_col:
    fifa_small = fifa_small.rename(columns={fifa_points_col: "fifa_points"})
if fifa_date_col:
    fifa_small = fifa_small.rename(columns={fifa_date_col: "rank_date"})
    fifa_small["rank_date"] = safe_to_datetime(fifa_small["rank_date"])

# Normalizar tipos
fifa_small["team"] = fifa_small["team"].apply(norm_team_name)
fifa_small["fifa_rank"] = pd.to_numeric(fifa_small["fifa_rank"], errors="coerce")
if "fifa_points" in fifa_small.columns:
    fifa_small["fifa_points"] = pd.to_numeric(fifa_small["fifa_points"], errors="coerce")

# 7.2) Criar match_date (data do jogo ou fallback por year)
if "match_date" not in matches.columns:
    if date_col is not None and date_col in matches.columns:
        matches["match_date"] = safe_to_datetime(matches[date_col])
    elif "year" in matches.columns:
        matches["match_date"] = pd.to_datetime(matches["year"].astype(str) + "-07-01", errors="coerce")
    else:
        raise ValueError("Não existe date_col nem coluna 'year' para criar match_date.")

# fallback: preencher NaT usando year
if "year" in matches.columns:
    mask = matches["match_date"].isna() & matches["year"].notna()
    if mask.any():
        matches.loc[mask, "match_date"] = pd.to_datetime(
            matches.loc[mask, "year"].astype(str) + "-07-01",
            errors="coerce"
        )

print("NaT em match_date:", matches["match_date"].isna().sum(), "de", len(matches))

# 7.3) Escolher estratégia: as-of só se houver histórico de datas no FIFA
use_asof = False
if "rank_date" in fifa_small.columns:
    fifa_small["rank_date"] = pd.to_datetime(fifa_small["rank_date"], errors="coerce")
    fifa_small = fifa_small.dropna(subset=["rank_date"]).copy()
    n_dates = fifa_small["rank_date"].nunique(dropna=True)
    use_asof = n_dates >= 5
    print(f"[INFO] rank_date distintas no FIFA: {n_dates} -> use_asof={use_asof}")

# Helper: obter sempre a PRIMEIRA coluna quando há nomes duplicados
def _first_col_as_series(df: pd.DataFrame, colname: str) -> pd.Series:
    x = df.loc[:, colname]  # pode devolver Series ou DataFrame se houver duplicadas
    if isinstance(x, pd.DataFrame):
        return x.iloc[:, 0]
    return x

if use_asof:
    # 7.4) merge_asof robusto (sem problemas com colunas duplicadas)
    def merge_asof_team(df_matches: pd.DataFrame, team_col_name: str, prefix: str) -> pd.DataFrame:
        team_s = _first_col_as_series(df_matches, team_col_name)
        date_s = _first_col_as_series(df_matches, "match_date")

        tmp = pd.DataFrame({
            "team": team_s,
            "match_date": pd.to_datetime(date_s, errors="coerce")
        })
        tmp["__idx"] = df_matches.index

        tmp["team"] = tmp["team"].apply(norm_team_name)
        tmp2 = tmp.dropna(subset=["team", "match_date"]).copy()

        # merge_asof exige chave temporal monotónica
        tmp2 = tmp2.sort_values(["match_date", "team"])

        right_cols = ["team", "rank_date", "fifa_rank"]
        if "fifa_points" in fifa_small.columns:
            right_cols.append("fifa_points")

        right = fifa_small[right_cols].copy()
        right["rank_date"] = pd.to_datetime(right["rank_date"], errors="coerce")
        right = right.dropna(subset=["team", "rank_date"]).sort_values(["rank_date", "team"])

        out = pd.merge_asof(
            tmp2,
            right,
            left_on="match_date",
            right_on="rank_date",
            by="team",
            direction="backward",
            allow_exact_matches=True
        )

        out = out.set_index("__idx").sort_index()

        res = pd.DataFrame(index=df_matches.index)
        res[f"{prefix}_rank"] = out["fifa_rank"]
        res[f"{prefix}_points"] = out["fifa_points"] if "fifa_points" in out.columns else np.nan
        return res

    home_rank = merge_asof_team(matches, "home_team", "home")
    away_rank = merge_asof_team(matches, "away_team", "away")
    matches = pd.concat([matches, home_rank, away_rank], axis=1)

else:
    # 7.5) Snapshot merge
    fifa_snap = fifa_small.dropna(subset=["team", "fifa_rank"]).copy()
    fifa_snap = fifa_snap.sort_values("fifa_rank").drop_duplicates("team", keep="first")

    home_map = {"team": "home_team", "fifa_rank": "home_rank"}
    away_map = {"team": "away_team", "fifa_rank": "away_rank"}

    if "fifa_points" in fifa_snap.columns:
        home_map["fifa_points"] = "home_points"
        away_map["fifa_points"] = "away_points"

    matches = matches.merge(fifa_snap.rename(columns=home_map), on="home_team", how="left")
    matches = matches.merge(fifa_snap.rename(columns=away_map), on="away_team", how="left")

# 7.6) Features fortes
matches["rank_diff"] = matches["away_rank"] - matches["home_rank"]
if "home_points" in matches.columns and "away_points" in matches.columns:
    matches["points_diff"] = matches["home_points"] - matches["away_points"]
else:
    matches["points_diff"] = np.nan

# 7.7) Remover jogos sem ranking
before = len(matches)
matches = matches.dropna(subset=["home_rank", "away_rank"]).copy()
after = len(matches)
print(f"Após merge ranking: {matches.shape} | removidos {before-after} jogos sem ranking")

# Preview
cols_show = ["home_team", "away_team", "home_rank", "away_rank", "rank_diff"]
if "stage" in matches.columns: cols_show.append("stage")
if "y" in matches.columns: cols_show.append("y")
display(matches[cols_show].head(10))


Colunas duplicadas após rename: []
NaT em match_date: 0 de 852
[INFO] rank_date distintas no FIFA: 333 -> use_asof=True
Após merge ranking: (355, 32) | removidos 497 jogos sem ranking


,home_team,away_team,home_rank,away_rank,rank_diff,stage,y
464,Spain,Korea Republic,5.0,37.0,32.0,Group C,1
465,Germany,Bolivia,1.0,43.0,42.0,Group C,2
466,USA,Switzerland,23.0,12.0,-11.0,Group A,1
468,Colombia,Romania,17.0,7.0,-10.0,Group A,0
469,Belgium,Morocco,27.0,28.0,1.0,Group F,2
470,Norway,Mexico,6.0,16.0,10.0,Group E,2
471,Cameroon,Sweden,24.0,10.0,-14.0,Group B,1
472,Netherlands,Saudi Arabia,2.0,34.0,32.0,Group F,2
473,Brazil,Russia,3.0,19.0,16.0,Group B,2
474,Argentina,Greece,8.0,31.0,23.0,Group D,2


## 8) Dataset final (X, y)

Montagem do dataset final (`X`, `y`): seleção de variáveis, remoção de colunas não preditivas e verificação de consistência.


In [ ]:
# --------------------------
# 8) Dataset final (X, y)
# --------------------------
feature_cols_num = [
    "home_rank", "away_rank", "rank_diff",
    "home_champion_before", "away_champion_before"
]
if "points_diff" in matches.columns:
    feature_cols_num.append("points_diff")

feature_cols_cat = ["stage"]

X = matches[feature_cols_num + feature_cols_cat].copy()
y = matches["y"].astype(int).copy()

print("X shape:", X.shape, "| y distribution:", np.bincount(y))
display(X.head())


X shape: (355, 7) | y distribution: [113  87 155]


,home_rank,away_rank,rank_diff,home_champion_before,away_champion_before,points_diff,stage
464,5.0,37.0,32.0,0,0,18.0,Group C
465,1.0,43.0,42.0,0,0,25.0,Group C
466,23.0,12.0,-11.0,0,0,-7.0,Group A
468,17.0,7.0,-10.0,0,0,-4.0,Group A
469,27.0,28.0,1.0,0,0,0.0,Group F


## 9) Split treino/teste (por ano se existir)

Divisão treino/teste, preferindo separação por **ano** quando disponível para reduzir *data leakage* e simular generalização temporal.


In [ ]:
# --------------------------
# 9) Split treino/teste (por ano se existir) — robusto
# --------------------------
from sklearn.model_selection import train_test_split, GroupShuffleSplit
import pandas as pd
import numpy as np

# Garantir que não há colunas duplicadas (evita bugs estranhos)
matches = matches.loc[:, ~matches.columns.duplicated()].copy()

# Escolher a melhor coluna de ano disponível
year_for_split = None
if "year" in matches.columns:
    year_for_split = "year"
elif year_col is not None and year_col in matches.columns:
    year_for_split = year_col
elif "Year" in matches.columns:
    year_for_split = "Year"

# Garantir alinhamento de índices (X e y normalmente vêm do matches filtrado)
common_idx = X.index.intersection(y.index).intersection(matches.index)
X = X.loc[common_idx]
y = y.loc[common_idx]

if year_for_split is not None:
    # groups alinhado ao mesmo index de X/y
    groups = matches.loc[common_idx, year_for_split]

    # Converter com segurança
    groups = pd.to_numeric(groups, errors="coerce").fillna(-1).astype(int).values

    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
    train_idx, test_idx = next(gss.split(X, y, groups=groups))

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    print(f"Split por grupos usando coluna: {year_for_split}")
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
    )
    print("Split stratified (sem coluna de ano disponível)")

print("Train:", X_train.shape, "| Test:", X_test.shape)


Split por grupos usando coluna: year
Train: (246, 7) | Test: (109, 7)


## 10) Pré-processamento (Pipeline)

Definição do pré‑processamento com `Pipeline`/`ColumnTransformer`: escalonamento de numéricas, *one‑hot* de categóricas e imputação quando necessário.


In [ ]:
# --------------------------
# 10) Pré-processamento (Pipeline)
# --------------------------
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, feature_cols_num),
        ("cat", categorical_transformer, feature_cols_cat),
    ]
)

def eval_model(name, pipe, Xtr, ytr, Xte, yte):
    pipe.fit(Xtr, ytr)
    pred = pipe.predict(Xte)

    acc = accuracy_score(yte, pred)
    f1m = f1_score(yte, pred, average="macro")

    print(f"\n=== {name} ===")
    print("Accuracy:", round(acc, 4), "| F1-macro:", round(f1m, 4))
    print("Confusion matrix:\n", confusion_matrix(yte, pred))
    print("\nClassification report:\n", classification_report(yte, pred, digits=4))

    if hasattr(pipe, "predict_proba"):
        proba = pipe.predict_proba(Xte)
        try:
            ll = log_loss(yte, proba, labels=[0,1,2])
            print("LogLoss:", round(ll, 4))
        except Exception:
            pass

    return pipe


## 11) Modelos (baseline + 3)

Treino e avaliação de múltiplos modelos (baseline + 3 algoritmos). Comparação por métricas (accuracy/F1) e análise por matriz de confusão.


In [ ]:
# --------------------------
# 11) Modelos (baseline + 3)
# --------------------------
baseline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", DummyClassifier(strategy="most_frequent"))
])

logreg = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=2000))
])

rf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", RandomForestClassifier(
        n_estimators=400,
        random_state=RANDOM_SEED,
        class_weight="balanced_subsample"
    ))
])

gb = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", GradientBoostingClassifier(random_state=RANDOM_SEED))
])

_ = eval_model("Baseline (most frequent)", baseline, X_train, y_train, X_test, y_test)
_ = eval_model("Logistic Regression", logreg, X_train, y_train, X_test, y_test)
_ = eval_model("Random Forest", rf, X_train, y_train, X_test, y_test)
_ = eval_model("Gradient Boosting", gb, X_train, y_train, X_test, y_test)



=== Baseline (most frequent) ===
Accuracy: 0.4587 | F1-macro: 0.2096
Confusion matrix:
 [[ 0  0 30]
 [ 0  0 29]
 [ 0  0 50]]

Classification report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        30
           1     0.0000    0.0000    0.0000        29
           2     0.4587    1.0000    0.6289        50

    accuracy                         0.4587       109
   macro avg     0.1529    0.3333    0.2096       109
weighted avg     0.2104    0.4587    0.2885       109

LogLoss: 19.5099

=== Logistic Regression ===
Accuracy: 0.5046 | F1-macro: 0.4083
Confusion matrix:
 [[15  3 12]
 [11  2 16]
 [ 9  3 38]]

Classification report:
               precision    recall  f1-score   support

           0     0.4286    0.5000    0.4615        30
           1     0.2500    0.0690    0.1081        29
           2     0.5758    0.7600    0.6552        50

    accuracy                         0.5046       109
   macro avg     0.4181    0.4430

C:\Users\Utilizador\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Utilizador\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Utilizador\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"


=== Random Forest ===
Accuracy: 0.4495 | F1-macro: 0.356
Confusion matrix:
 [[15  4 11]
 [12  1 16]
 [16  1 33]]

Classification report:
               precision    recall  f1-score   support

           0     0.3488    0.5000    0.4110        30
           1     0.1667    0.0345    0.0571        29
           2     0.5500    0.6600    0.6000        50

    accuracy                         0.4495       109
   macro avg     0.3552    0.3982    0.3560       109
weighted avg     0.3926    0.4495    0.4035       109

LogLoss: 1.0973

=== Gradient Boosting ===
Accuracy: 0.3945 | F1-macro: 0.329
Confusion matrix:
 [[12  7 11]
 [14  2 13]
 [17  4 29]]

Classification report:
               precision    recall  f1-score   support

           0     0.2791    0.4000    0.3288        30
           1     0.1538    0.0690    0.0952        29
           2     0.5472    0.5800    0.5631        50

    accuracy                         0.3945       109
   macro avg     0.3267    0.3497    0.3290      

## 12) Tuning (GridSearch) no RandomForest

Otimização de hiperparâmetros com `GridSearchCV` (foco no Random Forest) para melhorar desempenho e justificar a escolha do modelo final.


In [ ]:
# --------------------------
# 12) Tuning (GridSearch) no RandomForest
# --------------------------
param_grid = {
    "clf__n_estimators": [300, 600],
    "clf__max_depth": [None, 10, 20],
    "clf__min_samples_split": [2, 5],
    "clf__min_samples_leaf": [1, 2],
}

grid = GridSearchCV(
    rf,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)
print("\nBest params:", grid.best_params_)
print("Best CV score (F1-macro):", round(grid.best_score_, 4))

best_model = grid.best_estimator_
_ = eval_model("Best Tuned RandomForest (test)", best_model, X_train, y_train, X_test, y_test)

joblib.dump(best_model, "best_worldcup_model.joblib")
print("\nModelo guardado: best_worldcup_model.joblib")


Fitting 3 folds for each of 24 candidates, totalling 72 fits

Best params: {'clf__max_depth': None, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2, 'clf__n_estimators': 300}
Best CV score (F1-macro): 0.4478

=== Best Tuned RandomForest (test) ===
Accuracy: 0.4954 | F1-macro: 0.4225
Confusion matrix:
 [[14  5 11]
 [12  4 13]
 [11  3 36]]

Classification report:
               precision    recall  f1-score   support

           0     0.3784    0.4667    0.4179        30
           1     0.3333    0.1379    0.1951        29
           2     0.6000    0.7200    0.6545        50

    accuracy                         0.4954       109
   macro avg     0.4372    0.4415    0.4225       109
weighted avg     0.4681    0.4954    0.4672       109

LogLoss: 1.058

Modelo guardado: best_worldcup_model.joblib


## 13) Preparar tabela de forças (Top 32 do ranking)

Construção de uma tabela de "força"/ranking para as 32 seleções melhor classificadas (base para a simulação de campeão).


In [ ]:
# --------------------------
# 13) Preparar tabela de forças (Top 32 do ranking)
# --------------------------
# (Se o teu FIFA tiver data, isto vai usar o "melhor" ranking disponível;
#  aqui assumimos snapshot para simulação.)

def build_strength_table_from_fifa(fifa_df):
    ccol = find_col(fifa_df, ["team", fifa_country_col, "country_full", "country", "name", "pais"])
    rcol = find_col(fifa_df, ["fifa_rank", fifa_rank_col, "rank", "ranking", "position"])
    pcol = find_col(fifa_df, ["fifa_points", fifa_points_col, "total_points", "points"])

    tmp = fifa_df.copy()
    tmp = tmp.rename(columns={ccol: "team", rcol: "rank"})
    tmp["team"] = tmp["team"].apply(norm_team_name)
    tmp["rank"] = pd.to_numeric(tmp["rank"], errors="coerce")

    if pcol:
        tmp = tmp.rename(columns={pcol: "points"})
        tmp["points"] = pd.to_numeric(tmp["points"], errors="coerce")
    else:
        tmp["points"] = np.nan

    tmp = tmp.dropna(subset=["team", "rank"]).sort_values("rank").drop_duplicates("team", keep="first")
    return tmp[["team", "rank", "points"]].reset_index(drop=True)

strength = build_strength_table_from_fifa(fifa_small.rename(columns={
    "team": "team",
    "fifa_rank": "rank",
    **({"fifa_points": "points"} if "fifa_points" in fifa_small.columns else {})
}))

teams32 = strength.sort_values("rank").head(32)["team"].tolist()
print("Equipas usadas (Top 32 FIFA rank):")
display(pd.DataFrame({"team": teams32}))


Equipas usadas (Top 32 FIFA rank):


,team
0,Brazil
1,Spain
2,Belgium
3,Argentina
4,Germany
5,France
6,Italy
7,Netherlands
8,Czechia
9,Uruguay


## 14) Funções de simulação P(campeão)

Definição de funções para simular confrontos e estimar probabilidades (ex.: `predict_proba`) e agregação para obter **P(campeão)**.


In [ ]:
# --------------------------
# 14) Funções de simulação P(campeão)
# --------------------------
# Cache global para acelerar a simulação (evita predict_proba repetido)
PROBA_CACHE = {}

def build_match_features(teamA, teamB, stage_name="Group Stage"):
    teamA = norm_team_name(teamA)
    teamB = norm_team_name(teamB)

    ra = strength.loc[strength["team"] == teamA, "rank"]
    rb = strength.loc[strength["team"] == teamB, "rank"]
    if ra.empty or rb.empty:
        return None

    home_rank = float(ra.iloc[0])
    away_rank = float(rb.iloc[0])

    row = {
        "home_rank": home_rank,
        "away_rank": away_rank,
        "rank_diff": away_rank - home_rank,
        "home_champion_before": int(teamA in champions_ever),
        "away_champion_before": int(teamB in champions_ever),
        "stage": stage_name
    }

    # points_diff se existir
    if "points" in strength.columns and strength["points"].notna().any():
        pa = strength.loc[strength["team"] == teamA, "points"]
        pb = strength.loc[strength["team"] == teamB, "points"]
        if not pa.empty and not pb.empty:
            row["points_diff"] = float(pa.iloc[0]) - float(pb.iloc[0])
        else:
            row["points_diff"] = np.nan
    else:
        row["points_diff"] = np.nan

    return row

def simulate_single_match(model, teamA, teamB, stage_name):
    # Nota: a ordem (teamA, teamB) importa (home vs away), por isso a chave é "ordenada"
    key = (norm_team_name(teamA), norm_team_name(teamB), stage_name)

    if key in PROBA_CACHE:
        proba = PROBA_CACHE[key]
    else:
        feat = build_match_features(teamA, teamB, stage_name=stage_name)
        if feat is None:
            # fallback se faltar ranking
            proba = np.array([0.35, 0.30, 0.35], dtype=float)
        else:
            Xrow = pd.DataFrame([feat])
            proba = model.predict_proba(Xrow)[0]  # labels [0,1,2]
        PROBA_CACHE[key] = proba

    return np.random.choice([0,1,2], p=proba)

def simulate_group(model, teams, stage_name="Group Stage"):
    pts = {t: 0 for t in teams}

    for i in range(len(teams)):
        for j in range(i+1, len(teams)):
            A, B = teams[i], teams[j]
            outcome = simulate_single_match(model, A, B, stage_name)
            if outcome == 2:
                pts[A] += 3
            elif outcome == 0:
                pts[B] += 3
            else:
                pts[A] += 1
                pts[B] += 1

    items = list(pts.items())
    np.random.shuffle(items)  # desempate aleatório
    items.sort(key=lambda x: x[1], reverse=True)
    return [t for t,_ in items]

def simulate_knockout(model, teams, round_name):
    winners = []
    for i in range(0, len(teams), 2):
        A, B = teams[i], teams[i+1]
        outcome = simulate_single_match(model, A, B, round_name)

        if outcome == 2:
            winners.append(A)
        elif outcome == 0:
            winners.append(B)
        else:
            # empate -> penáltis: ligeira vantagem para melhor rank
            ra = float(strength.loc[strength["team"] == A, "rank"].iloc[0])
            rb = float(strength.loc[strength["team"] == B, "rank"].iloc[0])
            pA = 0.5 + (rb - ra) * 0.005
            pA = min(max(pA, 0.35), 0.65)
            winners.append(A if np.random.rand() < pA else B)

    return winners

def simulate_worldcup_32(model, teams32):
    teams32 = [norm_team_name(t) for t in teams32]
    teams32 = teams32.copy()
    np.random.shuffle(teams32)

    groups = [teams32[i*4:(i+1)*4] for i in range(8)]
    qualified = []

    # 8 grupos -> top2
    group_orders = []
    for g in groups:
        order = simulate_group(model, g, "Group Stage")
        group_orders.append(order)
        qualified.extend(order[:2])

    # Oitavos: A1 vs B2; B1 vs A2; ...
    r16 = []
    for k in range(0, 8, 2):
        A = group_orders[k]
        B = group_orders[k+1]
        r16 += [A[0], B[1], B[0], A[1]]

    qf = simulate_knockout(model, r16, "Round of 16")
    sf = simulate_knockout(model, qf, "Quarter-finals")
    fin = simulate_knockout(model, sf, "Semi-finals")
    champ = simulate_knockout(model, fin, "Final")[0]
    return champ

def champion_probabilities(model, teams, n_sims=5000):
    counts = {t: 0 for t in teams}
    for _ in range(n_sims):
        champ = simulate_worldcup_32(model, teams)
        counts[champ] += 1

    out = pd.DataFrame({"team": list(counts.keys()), "wins": list(counts.values())})
    out["p_champion"] = out["wins"] / n_sims
    out = out.sort_values("p_champion", ascending=False).reset_index(drop=True)
    return out

## 15) Correr simulação e ver Top 10

Execução da simulação e apresentação do Top 10 de probabilidades de campeão (resultados resumidos para discussão).


In [ ]:
# --------------------------
# 15) Correr simulação e ver Top 10
# --------------------------
# best_model = joblib.load("best_worldcup_model.joblib")

import time

# Limpar cache de probabilidades 
try:
    PROBA_CACHE.clear()
except Exception:
    PROBA_CACHE = {}

# Para simulações, é mais rápido usar 1 core por chamada (com cache, o nº de chamadas é baixo)
try:
    best_model.named_steps["clf"].set_params(n_jobs=1)
except Exception:
    pass

t0 = time.time()
probs = champion_probabilities(best_model, teams32, n_sims=500)  # aumenta para 10000 para mais estabilidade
dt = time.time() - t0

print(f"Simulação concluída em {dt:.2f}s | chamadas únicas predict_proba: {len(PROBA_CACHE)}")
display(probs.head(10))


Simulação concluída em 99.29s | chamadas únicas predict_proba: 3472


,team,wins,p_champion
0,Portugal,65,0.130
1,Belgium,57,0.114
2,Netherlands,49,0.098
3,Argentina,35,0.070
4,Spain,34,0.068
5,Germany,29,0.058
6,Chile,24,0.048
7,Uruguay,19,0.038
8,Colombia,19,0.038
9,England,19,0.038


## 16) Ver probabilidade de um país específico

Consulta rápida da probabilidade de campeão para um país específico (função utilitária para explorar resultados).


In [ ]:
# --------------------------
# 16) Ver probabilidade de um país específico
# --------------------------
def prob_of_country(df_probs, country_name):
    country_name = norm_team_name(country_name)
    row = df_probs[df_probs["team"] == country_name]
    if row.empty:
        return None
    return float(row["p_champion"].iloc[0])

pais = teams32[0]
print(f"Probabilidade estimada de '{pais}' ser campeão:", prob_of_country(probs, pais))


Probabilidade estimada de 'Brazil' ser campeão: 0.026


## 17) Exportar resultados (opcional)

Exportação opcional dos resultados (CSV) para anexar ao relatório ou reutilizar noutros notebooks.


In [ ]:
# --------------------------
# 17) Exportar resultados (opcional)
# --------------------------
probs.to_csv("champion_probabilities_top32.csv", index=False)
print("Guardado: champion_probabilities_top32.csv")


Guardado: champion_probabilities_top32.csv


### Interpretação rápida (Notebook 1)

**Preparação dos dados**
- As colunas principais foram detetadas corretamente (FIFA: `country_full`, `rank`, `total_points`, `rank_date`; Matches: equipas, golos, `Stage`, `Year`, `Datetime`; Cups: `Winner`, `Year`).
- O target é desequilibrado: **HomeWin (2)** é o resultado mais frequente (~57%), seguido de **Draw (1)** (~22%) e **AwayWin (0)** (~20%).  
- Ao fazer o *merge* do ranking FIFA por data (*as-of*), muitos jogos ficaram sem ranking compatível e foram removidos, reduzindo o dataset efetivo (ex.: ficaram **355 jogos** após remover jogos sem ranking). Isto limita a generalização e torna a tarefa mais exigente.

**Métricas e avaliação**
- A **accuracy** por si só pode ser enganadora em classes desequilibradas, pois um modelo pode obter boa accuracy apenas por prever a classe mais comum.
- Por isso, a métrica principal usada é o **F1-macro**, que calcula o F1 por classe (Away/Draw/Home) e faz a média, dando **peso igual** a todas as classes. Assim avaliamos melhor o desempenho em empates e vitórias fora, que são tipicamente mais difíceis.

**Split treino/teste**
- O split foi feito **por grupos de ano** (GroupShuffleSplit) para reduzir o risco de *data leakage*. Jogos da mesma edição/ano podem partilhar padrões e contexto; separar por ano torna o teste mais realista, simulando edições “não vistas” no treino.

**Resultados dos modelos**
- O **baseline (classe mais frequente)** consegue alguma accuracy, mas tem **F1-macro baixo**, porque ignora as classes minoritárias.
- A **Logistic Regression** apresentou um desempenho equilibrado e interpretável, melhorando claramente face ao baseline.
- O **RandomForest afinado** obteve o melhor desempenho global em **F1-macro**, mas continua a evidenciar dificuldade na classe **Draw (1)** (recall mais baixo), o que é esperado dada a natureza imprevisível dos empates e o desequilíbrio das classes.

**Simulação de probabilidade de campeão (Top 32)**
- A simulação (Monte Carlo) usa as probabilidades do modelo para gerar muitos torneios e estimar a probabilidade de cada seleção ser campeã.
- Estes valores devem ser interpretados como **estimativas condicionadas ao modelo e às features usadas** (ranking, fase e histórico), não como previsões determinísticas.
- Aumentar `n_sims` torna os resultados mais estáveis, mas aumenta o tempo de execução.

### Lições aprendidas

- A **qualidade/consistência dos dados** (nomes de equipas, datas e chaves de junção) influencia diretamente a junção com o ranking e, por consequência, o desempenho do modelo.
- Como existe **desbalanceamento** entre classes (mais vitórias da equipa da casa), usar **F1-macro** ajuda a avaliar melhor a performance global do classificador.
- A otimização de hiperparâmetros melhora resultados, mas o ganho é limitado pela **quantidade de jogos utilizáveis** após remover partidas sem ranking.